In [1]:
from pathlib import Path

# Directories of interest
# include folders that you might want the llm to have access to
DIRECTORIES = [r"C:\\Users\\Jamin Carter\\Downloads\\web_archive", r"D:\Project\Web-scraping\TestFolder"]
    
    
files_to_vectorize=[]
for dir in DIRECTORIES:
    files = Path(dir).glob("*")
    for file_path in files:
        if file_path.is_file():
            with open(file_path, 'r', encoding='utf-8') as f:
                files_to_vectorize.append(str(file_path))

In [2]:
print(f"Total files to vectorize: {len(files_to_vectorize)}")

Total files to vectorize: 73


In [3]:
files_to_vectorize[7][55:65]

'_game8_co_'

In [4]:
# DIRECTORY SETUP
import os
if not os.path.exists("./db/sqlite"):
    os.makedirs("./db/sqlite")
    

if not os.path.exists("./db/chroma"):
    os.makedirs("./db/chroma")

In [5]:
import chromadb
import sqlite3
import hashlib
from tqdm import tqdm
# --- SQLite ---
conn = sqlite3.connect("./db/sqlite/isVectorized.db")
cur = conn.cursor()
cur.execute("PRAGMA journal_mode=WAL;")
cur.execute("""
CREATE TABLE IF NOT EXISTS vectorized_files (
    file_id TEXT PRIMARY KEY,
    hash TEXT
)
""")
conn.commit()

def is_vectorized(file_id, file_hash) -> bool:
    cur.execute(
        "SELECT 1 FROM vectorized_files WHERE file_id=? AND hash=?",
        (file_id, file_hash)
    )
    return cur.fetchone() is not None


# --- Chroma ---
client = chromadb.PersistentClient(path="./db/chroma/")
collection = client.get_or_create_collection("ALL_TEXT_FILES")


def clean_content(text: str) -> str:
    if not text:
        return ""

    text = text.replace("\r\n", "\n").replace("\r", "\n")

    lines = []
    for line in text.split("\n"):
        line = line.strip()
        if line:
            lines.append(line)

    return "\n".join(lines)

def chunking(text: str, chunk_size: int, overlap: int) -> list[str]:
    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        end = min(start + chunk_size, text_length)
        chunk = text[start:end]
        chunks.append(chunk)

        if end == text_length:
            break

        start += chunk_size - overlap

    return chunks

def add_to_chroma(file_path: str):
    with open(file_path, "r", encoding="utf-8") as f:
        raw = f.read()

    content = clean_content(raw)
    file_hash = hashlib.md5(content.encode("utf-8")).hexdigest()

    if is_vectorized(file_path, file_hash):
        #tqdm.write(f"[DEV] Skipping AV:{file_path[50:70]}")
        return

    #tqdm.write(f"[DEV] Attempting AV:{file_path[50:70]}")

    chunks = chunking(content, 1000, 200)

    # 1️⃣ Add to Chroma FIRST
    for i, chunk in enumerate(chunks):
        collection.add(
            documents=[chunk],
            metadatas=[{"file_id": file_path}],
            ids=[f"{file_path}::chunk_{i}"]
        )

    # 2️⃣ Mark vectorized AFTER success
    cur.execute(
        """
        INSERT INTO vectorized_files (file_id, hash)
        VALUES (?, ?)
        ON CONFLICT(file_id) DO UPDATE SET
            hash = excluded.hash
        """,
        (file_path, file_hash)
    )
    conn.commit()


In [6]:
add_to_chroma(files_to_vectorize[12])


In [7]:
result = collection.get(
    limit=1,
    include=["documents", "metadatas"]
)
print(result)


{'ids': ['C:\\Users\\Jamin Carter\\Downloads\\web_archive\\2025-12-18__github_com_open_webui_open_webui.txt::chunk_0'], 'embeddings': None, 'documents': ["Skip to content\nYou signed in with another tab or window. Reload to refresh your session.\nYou signed out in another tab or window. Reload to refresh your session.\nYou switched accounts on another tab or window. Reload to refresh your session.\nDismiss alert\nopen-webui\nPublic\nSponsor\nWatch\nCouldn't load subscription status.\nRetry\nUh oh!\nThere was an error while loading. Please reload this page.\nFork\n16.6k\nFork your own copy of open-webui/open-webui\nForks could not be loaded\nLoading\nUh oh!\nThere was an error while loading. Please reload this page.\nStarred\n118k\nLoading\nUh oh!\nThere was an error while loading. Please reload this page.\nStar\n118k\nLoading\nUh oh!\nThere was an error while loading. Please reload this page.\nWatch536\nSponsor open-webui/open-webui\nSponsor open-webui/open-webui\nGitHub Sponsors\nLear

In [8]:
from tqdm import tqdm

pbar = tqdm(files_to_vectorize, desc="Vectorizing files")

for file_path in pbar:
    pbar.set_postfix(file=file_path[56:])  # show last part only
    add_to_chroma(file_path)


Vectorizing files: 100%|██████████| 73/73 [00:00<00:00, 861.83it/s, file=]                                                                          


In [9]:
def query_chroma(query: str, n_results: int = 5):
    results = collection.query(
        query_texts=[query],
        n_results=n_results,
        include=["documents", "metadatas"]
    )
    return results

query_chroma("What is web scraping?", n_results=3)["documents"]

[["traction engines (Tika, Docling, Document Intelligence, Mistral OCR, External loaders). Load documents directly into chat or add files to your document library, effortlessly accessing them using the # command before a query.\n🔍 Web Search for RAG: Perform web searches using 15+ providers including SearXNG, Google PSE, Brave Search, Kagi, Mojeek, Tavily, Perplexity, serpstack, serper, Serply, DuckDuckGo, SearchApi, SerpApi, Bing, Jina, Exa, Sougou, Azure AI Search, and Ollama Cloud, injecting results directly into your chat experience.\n🌐 Web Browsing Capability: Seamlessly integrate websites into your chat experience using the # command followed by a URL. This feature allows you to incorporate web content directly into your conversations, enhancing the richness and depth of your interactions.\n🎨 Image Generation & Editing Integration: Create and edit images using multiple engines including OpenAI's DALL-E, Gemini, ComfyUI (local), and AUTOMATIC1111 (local), with support for both gen

In [ ]:
import keyboard
import pyautogui
import pyperclip
import time
import tkinter as tk
from tkinter import simpledialog

# 1. Use the query function you already built in Cell 9
# (Ensure you've run Cell 9 before starting the listener)

SYSTEMPROMPT = """
### SYSTEM INSTRUCTIONS
You are a helpful AI assistant. Your primary task is to answer the user's query using the PROVIDED CONTEXT below. 
1. Always start your response by acknowledging the context (e.g., "According to the provided documents...").
2. If the answer is not in the context, explicitly state: "The provided context does not contain information about this, but based on general knowledge..."
3. Keep your response concise and directly related to the user's query.
4. If context is relevant but insufficient, combine it with your own knowledge to provide a comprehensive answer. Explicitly mention that the answer is based on your knowledge.

You will be provided with a Query and a set of documents from a vector database as context.
"""

def get_selected_text():
    """Tries to grab highlighted text from any window."""
    old_clipboard = pyperclip.paste()
    pyperclip.copy('') 
    pyautogui.hotkey('ctrl', 'c')
    time.sleep(0.15) 
    
    selected_text = pyperclip.paste().strip()
    if not selected_text:
        pyperclip.copy(old_clipboard)
        return None
    return selected_text

def trigger_ai_search():
    """Main logic that runs when you press the shortcut."""
    # Try to grab selection first
    user_query = get_selected_text()

    # Fallback to popup if nothing highlighted
    if not user_query:
        root = tk.Tk()
        root.withdraw()
        user_query = simpledialog.askstring("Query Chroma", "Enter your query:")
        root.destroy()
    
    if user_query:
        # IMPORTANT: We MUST call the query function here to get FRESH results
        # search_data will look like: {'documents': [[doc1, doc2]], ...}
        search_data = query_chroma(user_query, n_results=3) 
        
        formatted_response = SYSTEMPROMPT + f"\n### CONTEXT FROM VECTOR DB\nQuery: {user_query}\n\n"
        
        # FIX: We iterate over search_data['documents'][0]
        # This is a list of actual text blocks, not characters.
        for doc_text in search_data['documents'][0]:
            formatted_response += f"- {doc_text}\n\n"
            
        formatted_response += "--- END OF CONTEXT ---"

        pyperclip.copy(formatted_response)
        print(f"Done! Prompt for '{user_query[:20]}...' is in your clipboard.")

# Register the shortcut
keyboard.add_hotkey('ctrl+alt+v', trigger_ai_search)

print("Listener active. Press Ctrl+Alt+V to search ChromaDB...")
keyboard.wait()

Listener active. Press Ctrl+Alt+V to search ChromaDB...
Done! Prompt for 'What is the best wea...' is in your clipboard.
